In [2]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [3]:
##  Trans-ancestry

In [4]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)

In [9]:
lead <- read.table("../../ANALYSIS/lead_transancestry_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(TRANSANCESTRY_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [22]:
sumstat_variants <- data.frame()

for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".z")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>%select(MarkerName,Allele1,Allele2,Freq.TRANS,p,N) distinct(MarkerName,.keep_all = TRUE)
write.table(tmp_df,paste0("../../ANALYSIS/FINEMAP/trans/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
}

In [23]:
### European 

In [3]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUR_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)

In [4]:
lead <- read.table("../../ANALYSIS/lead_european_3mb_merged.txt",fill=TRUE,header=TRUE)
 output_df <- merge(EUR_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME.x,POSITION.x,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME.x,position=POSITION.x,allele1=METSIM_EA,allele2=METSIM_NEA,maf=MAF_AVG,beta=BETA_FE,se=STD_FE)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [5]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".z")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct(rsid,.keep_all = TRUE)
write.table(tmp_df,paste0("../../ANALYSIS/FINEMAP/eur/",file_name),sep = " ",row.names=FALSE,quote=FALSE)
}